## Import libs

In [14]:
import tensorflow as tf
from pathlib import Path
from util.dataset import Dataset
from util import read_data as rd
from tensorflow.keras import layers
from madras_laftr import trainer, tester
from madras_laftr.models import *
from util.results import ResultLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset
from util import metrics

## Preliminaries

In [2]:
batch_size = 64
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)
class_coeff = 1.0
fair_coeff = 1.0
recon_coeff = 0.0
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

## Load data

In [3]:
data_name = 'adult'

data_info = rd.return_data_info(data_name)
npzfile = rd.return_npz(data_name)

data = Dataset(npzfile=npzfile, name=data_name, a0_name=data_info['a0_name'], a1_name=data_info['a1_name'], 
                use_a=data_info['use_a'], seed=data_info['seed'], batch_size=batch_size)

data_shapes = list(data.get_shapes())
#print(data_shapes)

xdim = data_shapes[0][1]
ydim = data_shapes[1][1]
adim = data_shapes[2][1]
zdim = 8

In [5]:
train_data = Dataset.from_tensor_slices((data.x_train, data.y_train, data.a_train))
train_data = train_data.batch(batch_size)
train_data

<BatchDataset shapes: ((None, 113), (None, 1), (None, 1)), types: (tf.float64, tf.float32, tf.float32)>

In [22]:
valid_data = Dataset.from_tensor_slices((data.x_valid, data.y_valid, data.a_valid))
valid_data = valid_data.batch(batch_size)

## Train loop

In [10]:
def train(model, X, Y, A, optimizer, learning_rate=0.001):
    
    '''training enc-clas-dec'''
   
    enc_clas_dec = model.enc.variables + model.clas.variables + model.dec.variables
    adv = model.adv.variables
   
    with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
        
        tape0.watch(enc_clas_dec)
        tape1.watch(adv)

        model(X, Y, A) #to compute the foward
        current_loss = model.loss
    
    grads = tape0.gradient(current_loss, enc_clas_dec)
    optimizer.apply_gradients(zip(grads, enc_clas_dec))

    grads_adv = tape1.gradient(current_loss, adv)
    optimizer.apply_gradients(zip(grads_adv, adv))
    

In [21]:
def train_loop(model, train_dataset, epochs, optmizer):
    
    print("> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc")

    losses = {'clas':[], 'dec':[], 'adv':[], 'model':[]}
    accs = {'clas':[], 'dec':[], 'adv':[]}

    for epoch in range(epochs):
        
        for X, Y, A in train_dataset:
            
            train(model, X, Y, A, optmizer)

            losses['model'].append(model.loss)
            losses['clas'].append(model.class_loss)
            losses['dec'].append(model.recon_loss)
            losses['adv'].append(model.adv_loss)
            
            accs['clas'].append(metrics.accuracy(Y, model.Y_hat))
            accs['dec'].append(metrics.accuracy(X, model.X_hat))
            accs['adv'].append(metrics.accuracy(A, model.A_hat))

    
        print("> {} | {} | {} | {} | {} | {} | {} | {}".format(
            epoch+1, 
            losses['model'][-1], 
            tf.reduce_mean(losses['clas'][-1]), 
            tf.reduce_mean(losses['dec'][-1]), 
            tf.reduce_mean(losses['adv'][-1]),
            accs['clas'][-1], 
            accs['dec'][-1], 
            accs['adv'][-1]))

## Validation

In [30]:
def validation(model, valid_data):
    losses = {'clas':[], 'dec':[], 'adv':[], 'model':[]}
    accs = {'clas':[], 'dec':[], 'adv':[]}
    
    for X, Y, A in valid_data:
        
        model(X, Y, A)
        
        losses['model'].append(model.loss)
        losses['clas'].append(model.class_loss)
        losses['dec'].append(model.recon_loss)
        losses['adv'].append(model.adv_loss)
            
        accs['clas'].append(metrics.accuracy(Y, model.Y_hat))
        accs['dec'].append(metrics.accuracy(X, model.X_hat))
        accs['adv'].append(metrics.accuracy(A, model.A_hat))

    
    print("> Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc")
    print("> {} | {} | {} | {} | {} | {} | {}".format(
            losses['model'][-1], 
            tf.reduce_mean(losses['clas'][-1]), 
            tf.reduce_mean(losses['dec'][-1]), 
            tf.reduce_mean(losses['adv'][-1]),
            accs['clas'][-1], 
            accs['dec'][-1], 
            accs['adv'][-1]))

    return losses, accs

## Models

### For DP

In [16]:
model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs)

In [19]:
train_loop(model, train_data, 2, opt)

Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.4501360356807709 | 1.3504080772399902 | 44069.859375 | 2.7300333976745605 | 0.71875 | -107.703125 | 0.5625
> 2 | 0.444221168756485 | 1.3326635360717773 | 44073.90625 | 2.6915078163146973 | 0.734375 | -107.421875 | 0.5625


In [31]:
valid_losses, valid_accs = validation(model, valid_data)

> Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 0.2625291347503662 | 0.7875874042510986 | 54112.0859375 | 3.5792055130004883 | 0.84375 | -131.328125 | 0.546875


### For EqOdds

### For EqOpp

In [ ]:
#pegar só y=0